# Package Generation

In [1]:
%load_ext literary.notebook

In [2]:
from pathlib import Path
from shutil import which
from typing import Iterable

import nbformat
from traitlets.config import Config

from .exporter import LiteraryPythonExporter

In [3]:
DEFAULT_IGNORE_PATTERNS = (".ipynb_checkpoints", "__pycache__", ".*")

In [4]:
def create_notebook_code(exporter, path: Path) -> str:
    """Return the source code for a given notebook file

    :param nb_exporter: nbconvert exporter instance
    :param path: path to notebook
    :return:
    """
    nb = nbformat.read(path, as_version=nbformat.NO_CONVERT)
    body, resources = exporter.from_notebook_node(nb)
    return body

In [5]:
def _build_package_component(
    exporter,
    source_dir_path: Path,
    dest_dir_path: Path,
    ignore_patterns: Iterable[str] = None,
):
    """Recursively build a pure-Python package from a source tree

    :param nb_exporter: nbconvert exporter instance
    :param source_dir_path: path to current source directory
    :param dest_dir_path: path to current destination directory
    :param ignore_patterns: glob patterns of files and directories to ignore during
    recursion
    :return:
    """
    if ignore_patterns is None:
        ignore_patterns = DEFAULT_IGNORE_PATTERNS

    for path in source_dir_path.iterdir():
        # Ignore any unwanted files or directories
        if any(path.match(p) for p in ignore_patterns):
            continue

        # Find equivalent path in generated package
        relative_path = path.relative_to(source_dir_path)
        mirror_path = dest_dir_path / relative_path

        # Rewrite notebook in target directory
        if path.match("*.ipynb"):
            source = create_notebook_code(exporter, path)
            mirror_path.with_suffix(".py").write_text(source)

        # Recurse into directory
        elif path.is_dir():
            mirror_path.mkdir(parents=True, exist_ok=True)
            build_package_component(nb_exporter, path, mirror_path)

        # Copy file directly
        else:
            mirror_path.write_bytes(
                path.read_bytes(),
            )

In [6]:
def build_packages(
    source_path: Path,
    dest_path: Path,
    ignore_patterns: Iterable[str] = None,
):
    """Build a pure-Python package from a literary source tree

    :param source_path: path to directory containing notebook package directories
    :param dest_path: path to directory containing generated packages
    :param ignore_patterns: glob patterns of files and directories to ignore during
    recursion
    :return:
    """
    exporter = LiteraryPythonExporter(config=Config())

    if not source_path.exists():
        raise FileNotFoundError(f"Source path {source_path!r} does not exist")
    if not dest_path.exists():
        raise FileNotFoundError(f"Destination path {dest_path!r} does not exist")

    # Empty destination contents
    for p in dest_path.iterdir():
        shutil.rmtree(p)

    _build_package_component(exporter, source_path, dest_path, ignore_patterns)